In [2]:
graphics = False

In [3]:
def main():
    with open("input.txt") as file:
        data = file.read().strip()
    if "\n" in data:
        data = data.split("\n")
    data = process_input(data)
    part1 = evalp1(data, graphics)
    print("Part 1:")
    print(part1[0])
    print("Part 2:")
#    print(evalp2(data, part1[1], graphics))

In [4]:
def process_input(inp):
    boss = {}
    for line in inp:
        line = line.split(": ")
        boss[line[0][:3].lower()] = int(line[1])
    return (boss['hit'], boss['dam'])

In [5]:
def evalp1(data, graphics):
    spells = {"Magic Missile": {"cost": 53,  "damage": 4},
              "Drain":         {"cost": 73,  "damage": 2, "hp": 2},
              "Shield":        {"cost": 113, "turns": 6, "armor": 7,  "constant": False},
              "Poison":        {"cost": 173, "turns": 6, "damage": 3, "constant": True},
              "Recharge":      {"cost": 229, "turns": 5, "mana": 101, "constant": True}}
    stages = [(data, (50, 0, 500, {}), 0)]
    next_stages = []
    playerturn = True
    while True:
        for stage in stages:
            if stage[0] == True: continue
            boss, player, spent = stage
            for name, spell in spells.items():
                if name in player[3].keys(): continue
                spent += spell["cost"]
                turn = battle(boss, player, {name: spell}, playerturn) + (spent,)
                if turn[0] and not turn in next_stages: next_stages.append(turn)
        playerturn = not playerturn
        if all(x[0] == True for x in next_stages): return (min(list(zip(*next_stages))[1]), spells)
        stages = next_stages
        next_stages = []

In [ ]:
def evalp2(data, part1, graphics):
    pass

In [6]:
def battle(boss, player, spell, playerturn):
    bh, bd = boss
    ph, pa, mana, effects = player
    add_effect = {}
    if playerturn:
        cast = list(spell.values())[0]
        if "turns" in cast:
            add_effect = spell
            if not cast["constant"]:
                pa += cast["armor"] #if "armor" in cast
        else:
            if "damage" in cast:
                bh -= cast["damage"] #if "damage" in cast
                if bh <= 0: return (True,)
            if "hp" in cast: ph += cast["hp"]
    end_effects = []
    for name, e in effects.items():
        if e["constant"]:
            if "damage" in e:
                bh -= e["damage"]
                if bh <= 0: return (True,)
            if "mana" in e:
                mana += e["mana"]
        e["turns"] -= 1
        if e["turns"] == 0:
            if "armor" in e: pa -= e["armor"]
            end_effects.append(name)
    for name in end_effects: del effects[name]
    effects.update(add_effect)
    if not playerturn:
        damage = bd - pa
        if damage < 1: damage = 1
        ph -= damage
        if ph <= 0: return (False,)
    return ((bh, bd), (ph, pa, mana, effects))

In [7]:
main()

Part 1:
1459
Part 2:
